In [ ]:
# ============================================================
# CÉLULA 1 - Montar Google Drive
# ============================================================
from google.colab import drive
import os


drive.mount('/content/drive')
project_dir = "/content/drive/MyDrive/yolo_project"
os.makedirs(project_dir, exist_ok=True)
%cd $project_dir

Mounted at /content/drive
/content/drive/MyDrive/yolo_project


In [ ]:
# CÉLULA 2 - Instalar dependências
# ============================================================
!pip install -U pip
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 27.3 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 28.9 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [ultralytics]


In [ ]:
# ============================================================
# CÉLULA 3 - Baixar e extrair o dataset COCO128 (opcional)
# ============================================================
import zipfile

# Baixar COCO128
!curl -L https://github.com/ultralytics/yolov5/releases/download/v1.0/coco128.zip -o coco128.zip

# Extrair COCO128
with zipfile.ZipFile('coco128.zip', 'r') as zip_ref:
    zip_ref.extractall(project_dir)

coco_path = os.path.join(project_dir, 'coco128')
print(f"✅ Dataset COCO128 extraído em: {coco_path}")


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 6819k  100 6819k    0     0   9.9M      0 --:--:-- --:--:-- --:--:--  9.9M
✅ Dataset COCO128 extraído em: /content/drive/MyDrive/yolo_project/coco128


In [ ]:
# Célula 4 cria as pastas e separa 20% para validação
import os
import shutil
import random

coco_path = "/content/drive/MyDrive/yolo_project/coco128"
train_dir = os.path.join(coco_path, "images/train2017")
val_dir = os.path.join(coco_path, "images/val")
os.makedirs(val_dir, exist_ok=True)

# Lista todas as imagens
images = [f for f in os.listdir(train_dir) if f.endswith(".jpg")]
random.shuffle(images)

# Separar 20% para validação
num_val = int(0.2 * len(images))
val_images = images[:num_val]

# Mover imagens para a pasta val
for img in val_images:
    shutil.move(os.path.join(train_dir, img), os.path.join(val_dir, img))

print("✅ Separação treino/validação concluída")


✅ Separação treino/validação concluída


In [ ]:
# ============================================================
# CÉLULA 5 - Criar dataset.yaml corretamente
# ============================================================
coco_path = "/content/drive/MyDrive/yolo_project/coco128"
yaml_path = os.path.join(coco_path, 'dataset.yaml')

with open(yaml_path, 'w') as f:
    f.write(f"""train: {coco_path}/images/train2017
val: {coco_path}/images/val

nc: 80
names: ['person','bicycle','car','motorcycle','airplane','bus','train','truck','boat',
        'traffic light','fire hydrant','stop sign','parking meter','bench','bird','cat',
        'dog','horse','sheep','cow','elephant','bear','zebra','giraffe','backpack','umbrella',
        'handbag','tie','suitcase','frisbee','skis','snowboard','sports ball','kite',
        'baseball bat','baseball glove','skateboard','surfboard','tennis racket','bottle',
        'wine glass','cup','fork','knife','spoon','bowl','banana','apple','sandwich','orange',
        'broccoli','carrot','hot dog','pizza','donut','cake','chair','couch','potted plant',
        'bed','dining table','toilet','tv','laptop','mouse','remote','keyboard','cell phone',
        'microwave','oven','toaster','sink','refrigerator','book','clock','vase','scissors',
        'teddy bear','hair drier','toothbrush']""")

print(f"✅ dataset.yaml criado corretamente em: {yaml_path}")


✅ dataset.yaml criado corretamente em: /content/drive/MyDrive/yolo_project/coco128/dataset.yaml


In [ ]:
# ============================================================
# CÉLULA 6 - Treinar YOLOv8
# ============================================================
from ultralytics import YOLO

model = YOLO('yolov8n.pt')  # modelo pequeno e rápido

results = model.train(
    data=yaml_path,
    epochs=10,
    imgsz=640,
    batch=16,
    project=project_dir,
    name='coco128'
)


Ultralytics 8.3.195 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/yolo_project/coco128/dataset.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=coco1282, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, pl

/usr/local/lib/python3.12/dist-packages/ultralytics/utils/metrics.py:850: RuntimeWarning: Mean of empty slice.
  i = smooth(f1_curve.mean(0), 0.1).argmax()  # max F1 index
/usr/local/lib/python3.12/dist-packages/numpy/_core/_methods.py:130: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
       2/10      2.66G      1.198      1.372      1.213         72        640: 100% ━━━━━━━━━━━━ 7/7 4.7it/s 1.5s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 1/1 6.4it/s 0.2s
                   all         25          0          0          0          0          0
WARNING ⚠️ no labels found in detect set, can not compute metrics without labels


/usr/local/lib/python3.12/dist-packages/ultralytics/utils/metrics.py:850: RuntimeWarning: Mean of empty slice.
  i = smooth(f1_curve.mean(0), 0.1).argmax()  # max F1 index
/usr/local/lib/python3.12/dist-packages/numpy/_core/_methods.py:130: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
       3/10      2.66G       1.15      1.336      1.178         58        640: 100% ━━━━━━━━━━━━ 7/7 3.9it/s 1.8s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 1/1 4.3it/s 0.2s
                   all         25          0          0          0          0          0
WARNING ⚠️ no labels found in detect set, can not compute metrics without labels


...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
      10/10      2.67G      1.117      1.116      1.163         23        640: 100% ━━━━━━━━━━━━ 7/7 3.6it/s 1.9s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 1/1 3.1it/s 0.3s
                   all         25          0          0          0          0          0
WARNING ⚠️ no labels found in detect set, can not compute metrics without labels


/usr/local/lib/python3.12/dist-packages/ultralytics/utils/metrics.py:850: RuntimeWarning: Mean of empty slice.
  i = smooth(f1_curve.mean(0), 0.1).argmax()  # max F1 index
/usr/local/lib/python3.12/dist-packages/numpy/_core/_methods.py:130: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(



10 epochs completed in 0.007 hours.
Optimizer stripped from /content/drive/MyDrive/yolo_project/coco1282/weights/last.pt, 6.5MB
Optimizer stripped from /content/drive/MyDrive/yolo_project/coco1282/weights/best.pt, 6.5MB

Validating /content/drive/MyDrive/yolo_project/coco1282/weights/best.pt...
Ultralytics 8.3.195 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,151,904 parameters, 0 gradients, 8.7 GFLOPs
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 1/1 5.7it/s 0.2s


/usr/local/lib/python3.12/dist-packages/ultralytics/utils/metrics.py:668: RuntimeWarning: Mean of empty slice.
  ax.plot(px, py.mean(1), linewidth=3, color="blue", label=f"all classes {ap[:, 0].mean():.3f} mAP@0.5")
/usr/local/lib/python3.12/dist-packages/numpy/_core/_methods.py:138: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.12/dist-packages/ultralytics/utils/metrics.py:713: RuntimeWarning: Mean of empty slice.
  y = smooth(py.mean(0), 0.1)
/usr/local/lib/python3.12/dist-packages/numpy/_core/_methods.py:130: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(
/usr/local/lib/python3.12/dist-packages/ultralytics/utils/metrics.py:713: RuntimeWarning: Mean of empty slice.
  y = smooth(py.mean(0), 0.1)
/usr/local/lib/python3.12/dist-packages/numpy/_core/_methods.py:130: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(
/usr/local/lib/python3.12/dist-packages/ultraly

                   all         25          0          0          0          0          0
WARNING ⚠️ no labels found in detect set, can not compute metrics without labels
Speed: 0.2ms preprocess, 3.1ms inference, 0.0ms loss, 1.8ms postprocess per image
Results saved to /content/drive/MyDrive/yolo_project/coco1282


In [ ]:
# ============================================================
# CÉLULA 7 - Fazer inferência em imagens de teste
# ============================================================
from google.colab import files

print("➡️ Selecione imagens do seu computador para teste.")
uploaded_test = files.upload()

# Pasta para salvar imagens de teste
test_dir = os.path.join(project_dir, "test_images")
os.makedirs(test_dir, exist_ok=True)

for filename in uploaded_test.keys():
    with open(os.path.join(test_dir, filename), 'wb') as f:
        f.write(uploaded_test[filename])

# Caminho do modelo treinado
model_path = os.path.join(project_dir, "coco1282", "weights", "best.pt")

# Rodar a inferência
!yolo detect predict model={model_path} source={test_dir} save=True

print("✅ Inferência concluída! As imagens com predições foram salvas no Drive.")


➡️ Selecione imagens do seu computador para teste.


Saving GOPR8805.JPG to GOPR8805.JPG
Ultralytics 8.3.195 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,151,904 parameters, 0 gradients, 8.7 GFLOPs

image 1/1 /content/drive/MyDrive/yolo_project/test_images/GOPR8805.JPG: 480x640 9 persons, 1 handbag, 74.7ms
Speed: 5.0ms preprocess, 74.7ms inference, 154.1ms postprocess per image at shape (1, 3, 480, 640)
Results saved to /content/drive/MyDrive/yolo_project/runs/detect/predict
💡 Learn more at https://docs.ultralytics.com/modes/predict
✅ Inferência concluída! As imagens com predições foram salvas no Drive.
